# 데이터 준비하기: 다듬기, 변형, 병합

데이터 분석과 모델링 작업은 데이터를 불러오고, 다듬고, 변형하고 재정렬하는, 데이터 준비 과정에 많은 시간을 사용한다.

## 7.1. 데이터 합치기

pandas 객체에 저장된 데이터는 여러 내장 함수를 이용해서 합칠 수 있다.

* pandas.merge - sql 의 join 과 유사
* pandas.concat - 하나의 축에 따라 객체를 이어 붙인다
* combine_first - 두 객체를 포개서 한 객체에서 누락된 데이터를 다른 객체에 있는 값으로 채울 수 있도록 한다

### 7.1.1. 데이터베이스 스타일로 DataFrame 합치기

In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

In [2]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
               'data1': range(7)})
df2 = DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})

In [3]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [4]:
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [5]:
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


merge 수행시 어떤 key 값을 기준으로 수행할지 지정하지 않았는데, 기본값은 같은 key 이름을 기준으로 수행한다.<br/>
위와 같은 예에서는 "key" 가 될 수 있다.

In [6]:
pd.merge(df1, df2, on='key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


만약 동일한 key 이름을 갖고 있지 않다면, 지정해 주면 된다.

In [7]:
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                'data1': range(7)})
df4 = DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})

In [8]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


In [9]:
pd.merge(df1, df2, how='outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


M:N 관계도 병합이 가능하나, 직관적이진 않다.

In [12]:
df1 = DataFrame({'key':['b', 'b', 'a', 'c' , 'a', 'b'],
                'data1': range(6)})

In [13]:
df2 = DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                'data2':range(5)})

In [14]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [15]:
df2

,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


M:N 관계에서 조인은 두 로우의 데카르트 곱을 반환한다.<br/>
df1 에는 3개의 'b' 로우가 있고 오른쪽에는 2개의 'b' 로우가 있는데, 결과는 6개의 'b' 로우가 된다

In [16]:
pd.merge(df1, df2, how='inner')

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


여러 개의 키를 병합하려면 칼럼 이름이 들어간 리스트를 넘기면 된다.

In [17]:
left = DataFrame({'key1':['foo', 'foo', 'bar'],
                 'key2': ['one', 'two', 'own'],
                 'lval': [1, 2, 3]})

In [18]:
right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                  'key2': ['one', 'one', 'one', 'two'],
                  'rval': [4, 5, 6, 7]})

In [19]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,own,3.0,NaN
4,bar,one,NaN,6.0
5,bar,two,NaN,7.0


두 DataFrame 에 대해 merge 연산을 수행할 때 동일한 이름의 컬럼이 있다면 어떻게 해야 할까?<br/>
merge 함수에는 suffixes 라는 인자를 지원한다

In [20]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,own,3,one,6
5,bar,own,3,two,7


In [21]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,own,3,one,6
5,bar,own,3,two,7


merge 함수 인자 목록

<table align="left">
    <thead>
        <td>인자</td>
        <td>설명</td>
    </thead>
    <tbody>
        <tr>
            <td>left</td>
            <td>merge 하려는 DataFrame 중 왼쪽에 위치한 DataFrame</td>
        </tr>
        <tr>
            <td>right</td>
            <td>merge 하려는 DataFrame 중 오른쪽에 위치한 DataFrame</td>
        </tr>
        <tr>
            <td>how</td>
            <td>조인방법. 'inner'(기본값), 'outer', 'left', 'right'</td>
        </tr>
        <tr>
            <td>on</td>
            <td>조인하려는 로우 이름, 반드시 두 DataFrame 객체 모두에 있는 이름이어야 한다.<br/>만약 명시되지 않고 다른 조인 키도 주어지지 않으면 left와 right에서 공통되는 칼럼을 조인 키로 사용한다.</td>
        </tr>
        <tr>
            <td>left_on</td>
            <td>조인 키로 사용할 left DataFrame 의 칼럼</td>
        </tr>
        <tr>
            <td>right_on</td>
            <td>조인 키로 사용할 right DataFrame 의 칼럼</td>
        </tr>
        <tr>
            <td>left_index</td>
            <td>조인 키로 사용할 left DataFrame 의 색인 로우(다중 색인일 경우의 키)</td>
        </tr>
        <tr>
            <td>right_index</td>
            <td>조인 키로 사용할 right DataFrame 의 색인 로우(다중 색인일 경우의 키)</td>
        </tr>
        <tr>
            <td>sort</td>
            <td>조인 키에 따라 병합된 데이터를 사전 순으로 정렬. 기본 값은 True</td>
        </tr>
        <tr>
            <td>suffixes</td>
            <td>칼럼 이름이 겹칠 경우 각 칼럼 이름 뒤에 붙일 문자열의 튜플. 기본 값은 ('_x', '_y')</td>
        </tr>
        <tr>
            <td>copy</td>
            <td>False 일 경우 예외적인 경우에 결과로 데이터가 복사되지 않도록 한다. 기본 값은 항상 복사가 이루어진다.</td>
        </tr>
    </tbody>
</table>

### 7.1.2. 색인 머지하기

In [23]:
left1 = DataFrame({'key' : ['a', 'b', 'a', 'a', 'b', 'c'],
                  'value': range(6)})

In [24]:
right1 = DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [26]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [27]:
right1

,group_val
a,3.5
b,7.0


In [28]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [29]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


계층 색인 데이터

In [30]:
lefth = DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                  'key2': [2000, 2001, 2002, 2001, 2002],
                  'data': np.arange(5.)})

In [31]:
righth = DataFrame(np.arange(12).reshape((6,2)),
                  index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                        [2001, 2000, 2000, 2000, 2001, 2002]],
                  columns=['event1', 'event2'])

In [32]:
lefth

,data,key1,key2
0,0.0,Ohio,2000
1,1.0,Ohio,2001
2,2.0,Ohio,2002
3,3.0,Nevada,2001
4,4.0,Nevada,2002


In [33]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [34]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4,5
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11
3,3.0,Nevada,2001,0,1


In [35]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

,data,key1,key2,event1,event2
0,0.0,Ohio,2000.0,4.0,5.0
0,0.0,Ohio,2000.0,6.0,7.0
1,1.0,Ohio,2001.0,8.0,9.0
2,2.0,Ohio,2002.0,10.0,11.0
3,3.0,Nevada,2001.0,0.0,1.0
4,4.0,Nevada,2002.0,NaN,NaN
4,NaN,Nevada,2000.0,2.0,3.0


In [36]:
left2 = DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'],
                 columns=['Ohio', 'Nevada'])

In [37]:
right2 = DataFrame([[7., 8.], [9., 10.], [11., 12.], [13., 14.]],
                  index=['b', 'c', 'd', 'e'],
                  columns=['Missouri', 'Alabama'])

In [38]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [39]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [40]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [41]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [42]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


색인 대 색인으로 두 DataFrame 을 합치려면 그냥 간단히 머지하려는 DataFrame 의 리스트를 join 메서드에 넘기면 된다.

In [43]:
another = DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                   index=['a', 'c', 'e', 'f'],
                   columns=['New York', 'Oregon'])

In [44]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [45]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


### 7.1.3. 축 따라 이어붙이기

In [46]:
arr = np.arange(12).reshape((3,4))

In [47]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [48]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

Series 나 DataFrame 같은 pandas 객체의 컨텍스트 내부에는 축마다 이름이 있어서 배열을 쉽게 이어 붙일 수 있도록 되어 있다.

* 만약에 연결하려는 두 객체의 색인이 서로 다르다면 결과는 그 색인의 교집합이어야 하는가? 아니면 합집합이어야 하는가?
* 합쳐진 결과에서 합쳐지기 전 객체의 데이터를 구분할 수 있는가?
* 어떤 축으로 연결할 것인지 고려해야 하는가?

In [49]:
s1 = Series([0, 1], index=['a', 'b'])

In [50]:
s2 = Series([2, 3, 4], index=['c', 'd', 'e'])

In [51]:
s3 = Series([5, 6], index=['f', 'g'])

In [52]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [53]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [54]:
s4 = pd.concat([s1*5, s3])

In [55]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


In [56]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,5


In [57]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,5.0
e,NaN,NaN


Series 를 이어붙이기 전의 개별 series 를 구분할 수 없다는 문제가 생기는데, 이 경우 계층적 색인을 생성하여 식별이 가능하도록 할 수 있다.

In [58]:
result = pd.concat([s1, s2, s3], keys=['one', 'two', 'three'])

In [59]:
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [60]:
result.unstack()

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


In [61]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [63]:
df1 = DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'], columns=['one', 'two'])

In [64]:
df2 = DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'], columns=['three', 'four'])

In [65]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [66]:
pd.concat({'level1': df1, 'level2':df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [67]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

DataFrame의 로우 색인이 분석에 불필요한 경우에는 어떻게 할 것인가?<br/>
ignore_index=True 옵션을 이용하면 된다.

In [68]:
df1 = DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])

In [69]:
df2 = DataFrame(np.random.randn(2, 3), columns=['b', 'd' ,'e'])

In [70]:
df1

,a,b,c,d
0,-0.425672,-0.166342,0.036087,0.498812
1,-1.912918,-0.975561,-1.359159,-0.997910
2,-0.068930,2.015942,-0.997158,0.630499


In [71]:
df2

,b,d,e
0,0.068748,0.521448,-0.194877
1,0.047871,0.296370,1.726705


In [72]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d,e
0,-0.425672,-0.166342,0.036087,0.498812,NaN
1,-1.912918,-0.975561,-1.359159,-0.997910,NaN
2,-0.068930,2.015942,-0.997158,0.630499,NaN
3,NaN,0.068748,NaN,0.521448,-0.194877
4,NaN,0.047871,NaN,0.296370,1.726705


concat 함수 인자

<table align="left">
    <thead>
        <td>인자</td>
        <td>설명</td>
    </thead>
    <tbody>
        <tr>
            <td>objs</td>
            <td>이어붙일 pandas 객체의 사전이나 리스트, 필수인자</td>
        </tr>
        <tr>
            <td>axis</td>
            <td>이어붙일 축 방향, 기본 값은 0</td>
        </tr>
        <tr>
            <td>join</td>
            <td>조인 방식, 'inner'(내부조인, 교집합, 기본값) 와 'outer'(외부조인, 합집합) 가 있음</td>
        </tr>
        <tr>
            <td>join_axes</td>
            <td>합집합/교집합을 수행하는 대신 다른 n-1 축으로 사용할 색인을 지정한다.ㅏ</td>
        </tr>
        <tr>
            <td>keys</td>
            <td>이어붙일 객체나 이어붙인 축에 대한 계층 색인을  생성하는데 연관된 값이다. 리스트나 임의의 값이 들어있는 배열, 튜플의 배열 또는 배열의 리스트가 될 수 있다.</td>
        </tr>
        <tr>
            <td>levels</td>
            <td>계층 색인 레벨로 사용할 색인을 지정한다. keys 가 넘어온 경우 여러 개의 색인을 지정한다.</td>
        </tr>
        <tr>
            <td>names</td>
            <td>keys 나 levels 혹은 둘 다 있을 경우. 생성된 계층 레벨을 위한 이름</td>
        </tr>
        <tr>
            <td>verify_integrity</td>
            <td>이어붙인 객체에 중복되는 축이 있는지 검사하고 있다면 예외를 발생시킨다. 기본 값은 False로 중복을 허용한다</td>
        </tr>
        <tr>
            <td>ignore_index</td>
            <td>이어붙인 축의 색인을 유지하지 않고 range(total_length)로 새로운 색인을 생성한다</td>
        </tr>
    </tbody>
</table>

### 7.1.4. 겹치는 데이터 합치기

In [73]:
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
          index=['f', 'e', 'd', 'c', 'b', 'a'])

In [75]:
b = Series(np.arange(len(a), dtype=float), index=['f', 'e', 'd', 'c', 'b', 'a'])

In [76]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [77]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

In [78]:
np.where(pd.isnull(a), b, a)

array([ 0. ,  2.5,  2. ,  3.5,  4.5,  5. ])

combine_first() 라는 메서드는 위와 같은 연산을 제공하며, 데이터 정렬 기능까지 제공한다.

In [79]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

DataFrame 에서 combine_first() 메서드는 칼럼에 대해 같은 동작을 한다. 그래서 이를 통해 호출하는 객체에서 누락된 데이터를 인자로 넘긴 깨체에 있는 값으로 채워 넣을 수 있다.

In [80]:
df1 = DataFrame({'a': [1., np.nan, 5., np.nan],
                'b': [np.nan, 2., np.nan, 6.],
                'c': range(2, 18, 4)})

In [81]:
df2 = DataFrame({'a': [5., 4., np.nan, 3., 7.],
                'b': [np.nan, 3., 4., 6., 8.]})

In [83]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 7.2. 재형성과 피벗

표 형식의 데이터를 재배치하는 다양한 기본 연산이 존재하는데, 이런 연산을 재형성 또는 피벗 연산이라고 한다.

### 7.2.1. 계층적 색인으로 재형성하기

* stack : 데이터의 칼럼을 로우로 피벗 또는 회전시킨다.
* unstack : 로우를 칼럼으로 피벗시킨다.

In [84]:
data = DataFrame(np.arange(6).reshape((2,3)),
                index=pd.Index(['Ohio', 'Colorado'], name='state'),
                columns=pd.Index(['one', 'two', 'three'], name='number'))

In [85]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [86]:
result = data.stack()

In [87]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [88]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [89]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [90]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [91]:
s1 = Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])

In [92]:
s2 = Series([4, 5, 6], index=['c', 'd', 'e'])

In [93]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])

In [94]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [95]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [96]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [97]:
df = DataFrame({'left': result, 'right': result + 5},
              columns=pd.Index(['left', 'right'], name='side'))

In [98]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [99]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [100]:
df.unstack('state').stack('side')

state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

### 7.2.2. 피벗팅으로 데이터 나열 방식 바꾸기